In [1]:
import xmltodict
import pandas as pd
from convertbng.util import convert_bng, convert_lonlat

In [2]:
"""
Parses filepath (may be string or list) to get the image filename(s).
Returns list of filenames. 
"""
def parse_repro(repro_ref):
    if type(repro_ref) is str:
        repro_ref = [repro_ref]
        
    return [i.rsplit("\\", 1)[-1] for i in repro_ref if i != None]

In [3]:
"""
Gets stratigraphy type, de-dupes and returns as comma delimited string. 
Param strat_t might be dict or list of dicts.
"""
def parse_stratigraphy_type(strat_t):
    if strat_t == None:
        return strat_t
    if isinstance(strat_t, dict): 
        return strat_t['text']['#text']
    else:
        return ", ".join(set([parse_stratigraphy_type(x) for x in strat_t]))

In [4]:
"""
Generic list/str concatenation. 
Returns semicolon-delimited str. 
"""
def parse_list(item):
    try:
        result = item if type(item) is str else "; ".join(item)
    except TypeError:
        result = item
    finally:
        return result

In [5]:
"""
Conversion of OS BNG references to northing + easting
Adapted from: https://snorfalorpagus.net/blog/2014/08/12/converting-british-national-grid-and-irish-grid-references-a-practical-example/
"""
import re

# 5x5 grid letters, missing I
alphabet = 'ABCDEFGHJKLMNOPQRSTUVWXYZ'

def grid2xy(false_easting, false_northing, gridsizes, grid_ref):
    '''Convert grid reference to coordinates'''
    # false easting and northing
    easting = -false_easting
    northing = -false_northing

    # convert letter(s) to easting and northing offset
    for n in range(0, len(gridsizes)):
        letter = grid_ref[n]
        idx = alphabet.index(letter)
        col = (idx % 5)
        row = 4 - int((idx / 5))
        easting += (col * gridsizes[n])
        
        northing += (row * gridsizes[n])

  # numeric components of grid reference
    grid_ref = grid_ref[len(gridsizes):] # remove the letters
    e = '{:0<5}'.format(grid_ref[0:int(len(grid_ref)/2)]) 
    e = '{}.{}'.format(e[0:5],e[5:])
    
    n = '{:0<5}'.format(grid_ref[int(len(grid_ref)/2):])
    n = '{}.{}'.format(n[0:5],n[5:])
    
    easting += float(e)
    northing += float(n)
    
    return int(easting), int(northing)

def british2xy(grid_ref):
    # Filters out 'unknown' or other invalid values in OS field
    valid_ref = re.compile('[A-Z]{2}[0-9]+')
    if valid_ref.match(grid_ref) is None:
        return
    else:
        false_easting = 1000000
        false_northing = 500000
        gridsizes = [500000, 100000]
        return grid2xy(false_easting, false_northing, gridsizes, grid_ref)

In [6]:
# Read in adlib export xml, parse + write to dataframe

with open('input.xml') as fd:
    doc = xmltodict.parse(fd.read())

adlib_records = doc['adlibXML']['recordList']['record']
results = []

for n in adlib_records:
    print(n['priref'][0])
    results.append({
        'priref': n['priref'][0],
        'repro_ref': parse_repro(n['reproduction.reference']),
        'object_number': n['object_number'],
        'object_name': parse_list(n.get('object_name')),
        'os_gridref': n.get('field_coll.gridref', 'unknown'),
        'strat_unit': parse_list(n.get('stratigraphy.unit', [])),
        'strat_type': parse_stratigraphy_type(n.get('stratigraphy.type')),
        'taxonomy': parse_list(n.get('taxonomy.scientific_name')),
        'description': n['description'],
        'coll_name': parse_list(n.get('field_coll.name')),
        'coll_place': parse_list(n.get('field_coll.place')),
        'coords': british2xy(n.get('field_coll.gridref', 'unknown'))
    })

df = pd.DataFrame(results)

# Split coordinate tuple into easting + northing
df[['easting', 'northing']] = df['coords'].apply(pd.Series)
df.head()

51824
655139
51902
51903
54696
51907
51917
50807
51925
51963
51973
51976
52006
52007
52010
649642
52038
52039
52040
52041
52042
52043
52044
48976
48905
50924
50908
50923
52101
50621
52110
52359
50817
52363
52364
52453
52486
52835
653815
653816
654725
654727
52891
654084
653704
54507
655885
54490
655346
54805
655666
655377
80451
656823
651464
653851
654007
654008
54813
650236
54542
54563
655432
55344
657721
77209
655433
655434
655437
655438
655439
655440
655441
51575
51576
655442
658033
655443
660085
56947
57189
56946
51027
56943
56882
56879
56639
50811
48775
48776
48771
48489
48840
49097
48841
48833
48831
48832
75214
48237
48599
48620
48154
48685
48589
57670
48454
48467
47780
48608
48519
47796
48537
48782
49136
48694
49041
49095
656929
48203
48160
66434
66435
48784
49094
656926
656927
656928
655604
656414
53955
53768
53879
53706
53701
54137
52856
53029
53030
53031
53487
52626
52117
53990
52797
52798
52691
52732
47907
47916
53969
54021
48117
53897
54014
53931
53898
53237
53260
53066
657

656475
656476
656477
656478
656479
656480
656481
656482
656483
656486
659456
656491
656492
659428
659429
659430
656493
658919
659431
656488
656489
57567
57564
57696
57691
57867
57885
57941
58011
58043
58038
58052
58054
58055
71531
71555
71550
71566
71532
71534
71536
71537
71245
71246
71224
71226
71227
71239
71434
71436
71185
71175
71029
71020
71024
71108
71109
659186
659187
55135
55136
55137
55139
55157
55163
55167
55196
55203
55215
55216
55218
55221
55227
55230
55235
55260
657815
55264
55270
55278
55337
55473
55475
55491
55493
55497
55547
55612
55658
55674
55675
55683
55709
55715
55950
55951
55955
55981
55987
55996
56182
56192
56261
56267
56277
56318
56347
56348
56401
56408
56410
56446
56455
56473
56486
56491
56515
56518
56519
56625
56726
56807
56824
56826
56827
56828
56851
56855
56894
56900
56905
56937
56938
56955
55301
55302
55318
73149
73151
73152
73157
73158
73159
73160
73162
73167
73170
73172
73196
73197
73272
73278
73288
73318
73330
73331
73338
73341
73336
73337
73339
73345
7335

,coll_name,coll_place,coords,description,object_name,object_number,os_gridref,priref,repro_ref,strat_type,strat_unit,taxonomy,easting,northing
0,unknown,Marshwood; Marshbrook; Shropshire,"(344000, 289000)",Fossilised shell remains of the Ordovician bra...,Brachiopod,G.00055.001,SO4489,51824,[G.00055.001.jpg],"Chronostratigraphy, Lithostratigraphy",Ordovician; Caradoc Series; Marshbrookian Stag...,Dalmanella unguis,344000.0,289000.0
1,unknown,Marshwood; Marshbrook; Shropshire,"(344000, 289000)",Fossilised shell remains of the Ordovician bra...,Brachiopod,G.00055.002,SO4489,655139,[G.00055.002.jpg],"Chronostratigraphy, Lithostratigraphy",Ordovician; Caradoc Series; Marshbrookian Stag...,Dalmanella unguis,344000.0,289000.0
2,unknown,Marshbrook,"(344000, 289000)","Fossilised remains of Ordovician brachiopod, S...",Brachiopod,G.00093,SO4489,51902,[G.00093.jpg],Chronostratigraphy,Ordovician Period; Caradoc Series; Longvillian...,Strophomena grandis,344000.0,289000.0
3,unknown,Cheney Longville,"(341000, 285000)",Fossilised remains of single valve of Ordovici...,Brachiopod,G.00094,SO4185,51903,[G.00094.jpg],Chronostratigraphy,Ordovician Period; Caradoc Series; Aeronian St...,Kjaerina typa,341000.0,285000.0
4,unknown,Marshwood,None,Fossilised remains of shell of the Ordovician ...,Brachiopod,G.00096.002,unknown,54696,[G.00096.002.jpg],"Chronostratigraphy, Lithostratigraphy",Ordovician Period; Caradoc Series; Cheney Long...,Dalmanella wattsi,NaN,NaN


In [7]:
# Drop rows without coordinates
copy = df[['easting', 'northing']].dropna(subset=['easting'], how='all')
copy_e = list(copy['easting'].values)
copy_n = list(copy['northing'].values)

# Convert easting/northing to lat/long
copy['long'], copy['lat'] = convert_lonlat(copy_e, copy_n)
copy.head()

,easting,northing,long,lat
0,344000.0,289000.0,-2.826285,52.496032
1,344000.0,289000.0,-2.826285,52.496032
2,344000.0,289000.0,-2.826285,52.496032
3,341000.0,285000.0,-2.869758,52.459759
10,350000.0,274000.0,-2.735666,52.361775


In [8]:
# Stitch latlong back onto the main df and add a dummy column before retrieving image urls
df['lat'] = pd.Series(copy['lat'], index=df.index)
df['long'] = pd.Series(copy['long'], index=df.index)
df['url'] = 'NaN'

In [9]:
df.head()

,coll_name,coll_place,coords,description,object_name,object_number,os_gridref,priref,repro_ref,strat_type,strat_unit,taxonomy,easting,northing,lat,long,url
0,unknown,Marshwood; Marshbrook; Shropshire,"(344000, 289000)",Fossilised shell remains of the Ordovician bra...,Brachiopod,G.00055.001,SO4489,51824,[G.00055.001.jpg],"Chronostratigraphy, Lithostratigraphy",Ordovician; Caradoc Series; Marshbrookian Stag...,Dalmanella unguis,344000.0,289000.0,52.496032,-2.826285,NaN
1,unknown,Marshwood; Marshbrook; Shropshire,"(344000, 289000)",Fossilised shell remains of the Ordovician bra...,Brachiopod,G.00055.002,SO4489,655139,[G.00055.002.jpg],"Chronostratigraphy, Lithostratigraphy",Ordovician; Caradoc Series; Marshbrookian Stag...,Dalmanella unguis,344000.0,289000.0,52.496032,-2.826285,NaN
2,unknown,Marshbrook,"(344000, 289000)","Fossilised remains of Ordovician brachiopod, S...",Brachiopod,G.00093,SO4489,51902,[G.00093.jpg],Chronostratigraphy,Ordovician Period; Caradoc Series; Longvillian...,Strophomena grandis,344000.0,289000.0,52.496032,-2.826285,NaN
3,unknown,Cheney Longville,"(341000, 285000)",Fossilised remains of single valve of Ordovici...,Brachiopod,G.00094,SO4185,51903,[G.00094.jpg],Chronostratigraphy,Ordovician Period; Caradoc Series; Aeronian St...,Kjaerina typa,341000.0,285000.0,52.459759,-2.869758,NaN
4,unknown,Marshwood,None,Fossilised remains of shell of the Ordovician ...,Brachiopod,G.00096.002,unknown,54696,[G.00096.002.jpg],"Chronostratigraphy, Lithostratigraphy",Ordovician Period; Caradoc Series; Cheney Long...,Dalmanella wattsi,NaN,NaN,NaN,NaN,NaN


In [10]:
"""
Box integration: need to set up an app with OAuth 2.0 with JWT in advance.
Current app linked to s.vincent@nhm.ac.uk: https://nhm.app.box.com/developers/console/app/569998
Before each run + every 60 mins thereafter, need to generate a new temp access token + update app.cfg
"""
# Import two classes from the boxsdk module - Client and OAuth2
from boxsdk import Client, OAuth2

# Define client ID, client secret, and developer token.
CLIENT_ID = None
CLIENT_SECRET = None
ACCESS_TOKEN = None

# Read app info from text file
with open('app.cfg', 'r') as app_cfg:
    CLIENT_ID = app_cfg.readline()
    CLIENT_SECRET = app_cfg.readline()
    ACCESS_TOKEN = app_cfg.readline()


In [11]:
from boxsdk.network.default_network import DefaultNetwork

# Create OAuth2 object. It's already authenticated, thanks to the developer token.
oauth2 = OAuth2(CLIENT_ID, CLIENT_SECRET, access_token=ACCESS_TOKEN)

# Create the authenticated client
client = Client(oauth2, None)

In [20]:
# Access LudlowImages folder
folder_id = 48687719592

# Get metadata for image folder
ludlow = client.folder(folder_id).get()
limit = ludlow['item_collection']['total_count']

url_results = {}
folder = ludlow.get_items(offset=0, limit=limit)

# iterate over all the File objects in a folder and stash relevant metadata
for x in range(limit):
    f = folder.next()
    url_results[f.name] = (f.id, f.name, f.get_shared_link_download_url())
    print(f"{[x]} {f.name}")


[0] G.00055.001.jpg
[1] G.00055.002.jpg
[2] G.00093.jpg
[3] G.00094.jpg
[4] G.00096.002.jpg
[5] G.00106.jpg
[6] G.00113a.jpg
[7] G.00113b.jpg
[8] G.00118a.jpg
[9] G.00123a.jpg
[10] G.00123b.jpg
[11] G.00123c.jpg
[12] G.00133.jpg
[13] G.00143.jpg
[14] G.00145.jpg
[15] G.00147.jpg
[16] G.00148.jpg
[17] G.00151.jpg
[18] G.00160.jpg
[19] G.00182.jpg
[20] G.00183.jpg
[21] G.00184.jpg
[22] G.00185.jpg
[23] G.00186.jpg
[24] G.00187.jpg
[25] G.00188.jpg
[26] G.00191a.jpg
[27] G.00191b.jpg
[28] G.00192.001.jpg
[29] G.00194.jpg
[30] G.00196.jpg
[31] G.00197.jpg
[32] G.00202.jpg
[33] G.00214.jpg
[34] G.00216a.jpg
[35] G.00216b.jpg
[36] G.00267.jpg
[37] G.00269.jpg
[38] G.00272.jpg
[39] G.00273.jpg
[40] G.00309.jpg
[41] G.00317.jpg
[42] G.00380.jpg
[43] G.00386.jpg
[44] G.00387.jpg
[45] G.00400.001a.jpg
[46] G.00400.001b.jpg
[47] G.00400.002a.jpg
[48] G.00400.002b.jpg
[49] G.00404.jpg
[50] G.00408.002.jpg
[51] G.00436.jpg
[52] G.00443.jpg
[53] G.00468.jpg
[54] G.00471.jpg
[55] G.00483.jpg
[56] G.0

[439] G.03988.jpg
[440] G.03994.jpg
[441] G.03995.jpg
[442] G.03995a.jpg
[443] G.04013.jpg
[444] G.04015.jpg
[445] G.04024.jpg
[446] G.04025.jpg
[447] G.04057.jpg
[448] G.04060.jpg
[449] G.04066.jpg
[450] G.04070.jpg
[451] G.04071.jpg
[452] G.04072.jpg
[453] G.04075.jpg
[454] G.04090.jpg
[455] G.04090a.jpg
[456] G.04092.jpg
[457] G.04092b.jpg
[458] G.04092c.jpg
[459] G.04092d.jpg
[460] G.04092e.jpg
[461] G.04092f.jpg
[462] G.04092g.jpg
[463] G.04092h.jpg
[464] G.04106.jpg
[465] G.04108.jpg
[466] G.04109.jpg
[467] G.04116.001.jpg
[468] G.04116.002.jpg
[469] G.04128.jpg
[470] G.04137.001.jpg
[471] G.04137.002.jpg
[472] G.04137.003.jpg
[473] G.04137a.jpg
[474] G.04137b.jpg
[475] G.04137c.jpg
[476] G.04154.jpg
[477] G.04161.jpg
[478] G.04198.jpg
[479] G.04229.jpg
[480] G.04235.jpg
[481] G.04236a.jpg
[482] G.04236b.jpg
[483] G.04250.001.jpg
[484] G.04250.002.jpg
[485] G.04252.jpg
[486] G.04262a.jpg
[487] G.04262b.jpg
[488] G.04283.jpg
[489] G.04288.jpg
[490] G.04289.jpg
[491] G.04292.jpg
[4

[875] G.07783.jpg
[876] G.07795.jpg
[877] G.07803.jpg
[878] G.07807.jpg
[879] G.07819.002a.jpg
[880] G.07819.002b.jpg
[881] G.07819a.jpg
[882] G.07819b.jpg
[883] G.07825.jpg
[884] G.07827.001.jpg
[885] G.07827.002.jpg
[886] G.07838a.jpg
[887] G.07838b.jpg
[888] G.07843.jpg
[889] G.07852.jpg
[890] G.07860.jpg
[891] G.07861.jpg
[892] G.07866.001.jpg
[893] G.07866.002a.jpg
[894] G.07866.002b.jpg
[895] G.07866a.jpg
[896] G.07866b.jpg
[897] G.07876.jpg
[898] G.07877.002.jpg
[899] G.07881a.jpg
[900] G.07881b.jpg
[901] G.07884.jpg
[902] G.07887.001a.jpg
[903] G.07887.001b.jpg
[904] G.07898.001.jpg
[905] G.07905.001.jpg
[906] G.07906.001.jpg
[907] G.07906.002.jpg
[908] G.07906.jpg
[909] G.07911.jpg
[910] G.07921.002.jpg
[911] G.07921.002b.jpg
[912] G.07930.jpg
[913] G.07931.jpg
[914] G.07932.jpg
[915] G.07946.jpg
[916] G.07951.jpg
[917] G.07963.jpg
[918] G.07964.jpg
[919] G.07966.jpg
[920] G.07970.jpg
[921] G.07972.jpg
[922] G.07972b.jpg
[923] G.07973.jpg
[924] G.07973b.jpg
[925] G.08023.jpg
[

[1280] G.09908b.jpg
[1281] G.09908c.jpg
[1282] G.09909a.jpg
[1283] G.09909b.jpg
[1284] G.09909c.jpg
[1285] G.09921a.jpg
[1286] G.09921b.jpg
[1287] G.09923a.jpg
[1288] G.09923b.jpg
[1289] G.09923c.jpg
[1290] G.09927.001.jpg
[1291] G.09927.002.jpg
[1292] G.09932.jpg
[1293] G.09934.jpg
[1294] G.09940.002.jpg
[1295] G.09944.jpg
[1296] G.09946.001a.jpg
[1297] G.09946.001b.jpg
[1298] G.09946.002a.jpg
[1299] G.09946.002b.jpg
[1300] G.09950.001.jpg
[1301] G.09954.001a.jpg
[1302] G.09954.001b.jpg
[1303] G.09954.002a.jpg
[1304] G.09954.002b.jpg
[1305] G.09955a.jpg
[1306] G.09955b.jpg
[1307] G.09956.jpg
[1308] G.09957a.jpg
[1309] G.09957b.jpg
[1310] G.09960.jpg
[1311] G.09961.jpg
[1312] G.09962.001a.jpg
[1313] G.09962.001b.jpg
[1314] G.09962.002a.jpg
[1315] G.09962.002b.jpg
[1316] G.09962.002c.jpg
[1317] G.09968.002a.jpg
[1318] G.09968.002b.jpg
[1319] G.09977.jpg
[1320] G.09979.jpg
[1321] G.09986.jpg
[1322] G.10005.jpg
[1323] G.10022.jpg
[1324] G.10034.jpg
[1325] G.10037.jpg
[1326] G.10049.jpg
[1

[1682] G.12999a.jpg
[1683] G.12999b.jpg
[1684] G.13000.jpg
[1685] G.13000a.jpg
[1686] G.13004.jpg
[1687] G.13009.jpg
[1688] G.13011.001.jpg
[1689] G.13011.002.jpg
[1690] G.13011.003.jpg
[1691] G.13011.004.jpg
[1692] G.13011.005.jpg
[1693] G.13058a.jpg
[1694] G.13058b.jpg
[1695] G.13068.jpg
[1696] G.13073.jpg
[1697] G.13078.jpg
[1698] G.13091a.jpg
[1699] G.13100.jpg
[1700] G.13105.jpg
[1701] G.13108a.jpg
[1702] G.13108b.jpg
[1703] G.13111.jpg
[1704] G.13132.jpg
[1705] G.13136.jpg
[1706] G.13138.jpg
[1707] G.13145.jpg
[1708] G.13146.jpg
[1709] G.13148.jpg
[1710] G.13149.jpg
[1711] G.13153.jpg
[1712] G.13158.jpg
[1713] G.13162.jpg
[1714] G.13175a.jpg
[1715] G.13175b.jpg
[1716] G.13184.jpg
[1717] G.13186.001.jpg
[1718] G.13186.002.jpg
[1719] G.13189.jpg
[1720] G.13194.001.jpg
[1721] G.13194.002.jpg
[1722] G.13194.003.jpg
[1723] G.13206.jpg
[1724] G.13208.jpg
[1725] G.13219.jpg
[1726] G.13220.jpg
[1727] G.13222.jpg
[1728] G.13225.001a.jpg
[1729] G.13225.002.jpg
[1730] G.13226a.jpg
[1731] G.

[2096] G.15196.jpg
[2097] G.15201.jpg
[2098] G.15208a.jpg
[2099] G.15208b.jpg
[2100] G.15208c.jpg
[2101] G.15210.jpg
[2102] G.15217a.jpg
[2103] G.15217b.jpg
[2104] G.15217c.jpg
[2105] G.15220a.jpg
[2106] G.15220b.jpg
[2107] G.15220c.jpg
[2108] G.15228a.jpg
[2109] G.15228b.jpg
[2110] G.15228c.jpg
[2111] G.15229a.jpg
[2112] G.15229b.jpg
[2113] G.15229c.jpg
[2114] G.15230a.jpg
[2115] G.15230b.jpg
[2116] G.15230c.jpg
[2117] G.15231a.jpg
[2118] G.15231b.jpg
[2119] G.15231c.jpg
[2120] G.15233a.jpg
[2121] G.15233b.jpg
[2122] G.15233c.jpg
[2123] G.15238a.jpg
[2124] G.15238b.jpg
[2125] G.15238c.jpg
[2126] G.15503.jpg
[2127] G.15507.jpg
[2128] G.15518.jpg
[2129] G.15522.jpg
[2130] G.15525a.jpg
[2131] G.15525b.jpg
[2132] G.15526a.jpg
[2133] G.15574.002a.jpg
[2134] G.15574.002b.jpg
[2135] G.15580.001.jpg
[2136] G.15591.jpg
[2137] G.15617.jpg
[2138] G.15617b.jpg
[2139] G.15617c.jpg
[2140] G.15618.jpg
[2141] G.15618a.jpg
[2142] G.15618b.jpg
[2143] G.16001.jpg
[2144] G.16002.jpg
[2145] G.16043.jpg
[2

[2503] G.20196.jpg
[2504] G.20198.jpg
[2505] G.20199.jpg
[2506] G.20200.002.jpg
[2507] G.20200.003.jpg
[2508] G.20203.jpg
[2509] G.20205a.jpg
[2510] G.20205b.jpg
[2511] G.20206.jpg
[2512] G.20207.jpg
[2513] G.20208.jpg
[2514] G.20209.jpg
[2515] G.20211a.jpg
[2516] G.20211b.jpg
[2517] G.20212.jpg
[2518] G.20213.jpg
[2519] G.20214.jpg
[2520] G.20215.jpg
[2521] G.20216.jpg
[2522] G.20217.jpg
[2523] G.20218a.jpg
[2524] G.20218b.jpg
[2525] G.20220.jpg
[2526] G.20221.jpg
[2527] G.20222.jpg
[2528] G.20223a.jpg
[2529] G.20223b.jpg
[2530] G.20224.jpg
[2531] G.20224a.jpg
[2532] G.20224b.jpg
[2533] G.20226.jpg
[2534] G.20227.jpg
[2535] G.20229.001.jpg
[2536] G.20231.jpg
[2537] G.20232.jpg
[2538] G.20233.jpg
[2539] G.20234.jpg
[2540] G.20236.jpg
[2541] G.20239.jpg
[2542] G.20240.jpg
[2543] G.20240a.jpg
[2544] G.20240b.jpg
[2545] G.20242.jpg
[2546] G.20246.jpg
[2547] G.20247a.jpg
[2548] G.20247b.jpg
[2549] G.20248.jpg
[2550] G.20249.001.jpg
[2551] G.20249.002.jpg
[2552] G.20249.003.jpg
[2553] G.202

[2885] G.20506.007c.jpg
[2886] G.20507.002a.jpg
[2887] G.20507.002b.jpg
[2888] G.20507.002c.jpg
[2889] G.20507.002d.jpg
[2890] G.20507.002e.jpg
[2891] G.20508.002a.jpg
[2892] G.20508.002b.jpg
[2893] G.20508.002c.jpg
[2894] G.20508.002d.jpg
[2895] G.20508.002e.jpg
[2896] G.20509.002a.jpg
[2897] G.20509.002b.jpg
[2898] G.20509.002c.jpg
[2899] G.20510.001a.jpg
[2900] G.20510.001b.jpg
[2901] G.20510.002a.jpg
[2902] G.20510.002b.jpg
[2903] G.20510.002c.jpg
[2904] G.20510.003a.jpg
[2905] G.20510.003b.jpg
[2906] G.20510.003c.jpg
[2907] G.20511.002a.jpg
[2908] G.20511.002b.jpg
[2909] G.20511.002c.jpg
[2910] G.20512.002a.jpg
[2911] G.20513.001a.jpg
[2912] G.20513.001b.jpg
[2913] G.20513.002a.jpg
[2914] G.20513.002b.jpg
[2915] G.20513.002c.jpg
[2916] G.20513.002d.jpg
[2917] G.20513.002e.jpg
[2918] G.20514.001a.jpg
[2919] G.20514.001b.jpg
[2920] G.20514.002a.jpg
[2921] G.20514.002b.jpg
[2922] G.20514.002c.jpg
[2923] G.20514.002d.jpg
[2924] G.20514.002e.jpg
[2925] G.20514.006a.jpg
[2926] G.20514.0

[3226] shyms_g_1977_0604.002.jpg
[3227] shyms_g_1977_0607.jpg
[3228] shyms_g_1977_0615.jpg
[3229] shyms_g_1977_0623.001.jpg
[3230] shyms_g_1977_0623.002.jpg
[3231] shyms_g_1977_0623.003.jpg
[3232] shyms_g_1977_0625.jpg
[3233] shyms_g_1977_0635.jpg
[3234] shyms_g_1977_0637.jpg
[3235] shyms_g_1977_0649.jpg
[3236] shyms_g_1977_0650.jpg
[3237] shyms_g_1977_0665.jpg
[3238] shyms_g_1977_0685.jpg
[3239] shyms_g_1977_0700.jpg
[3240] shyms_g_1977_0701.001.jpg
[3241] shyms_g_1977_0701.002.jpg
[3242] shyms_g_1977_0711.001.jpg
[3243] shyms_g_1977_0711.002.jpg
[3244] shyms_g_1977_0711.003.jpg
[3245] shyms_g_1977_0722.jpg
[3246] shyms_g_1977_0747.jpg
[3247] shyms_g_1977_0750.001.jpg
[3248] shyms_g_1977_0750.002.jpg
[3249] shyms_g_1977_0755.jpg
[3250] shyms_g_1977_0758.001.jpg
[3251] shyms_g_1977_0758.003.jpg
[3252] shyms_g_1977_0805.jpg
[3253] shyms_g_1977_0806.jpg
[3254] shyms_g_1977_0835.jpg
[3255] shyms_g_1977_0847.jpg
[3256] shyms_g_1977_1565.jpg
[3257] shyms_g_1977_1566.jpg
[3258] shyms_g_1977_

In [21]:
# Get filename column and fill w dummy values
image_ref = pd.DataFrame(df['repro_ref'].copy())
image_ref['url'] = 'NaN'

# For every filename, for every row in table, attempt to match to filenames in box folder. 
for index, row in df.iterrows():
    for imagefile in row['repro_ref']:
        # If found, remove from url_results and either overwrite dummy value or concatenate (for > 1 image links)
        if imagefile in url_results:
            result = url_results[imagefile]
            if image_ref.loc[index]['url'] == 'NaN':
                image_ref.loc[index]['url'] = result[2]
            else: 
                image_ref.loc[index]['url'] = f"{image_ref.loc[index]['url']}; {result[2]}"
                
image_ref.head()

,repro_ref,url
0,[G.00055.001.jpg],https://nhm.box.com/shared/static/hk231of2dhxz...
1,[G.00055.002.jpg],https://nhm.box.com/shared/static/gqbj6h4qwtq8...
2,[G.00093.jpg],https://nhm.box.com/shared/static/oovi6on0t196...
3,[G.00094.jpg],https://nhm.box.com/shared/static/tivgnf50wm9e...
4,[G.00096.002.jpg],https://nhm.box.com/shared/static/xvx72pamptrz...


In [22]:
# Get all the records with at least one image and dump to csv for Portal upload
df['url'] = pd.Series(image_ref['url'], index=df.index)
df1 = df.drop(['coords', 'easting', 'northing'], axis=1).loc[lambda df: df.url != 'NaN']
df1.to_csv('ludlow_out.csv')

In [23]:
# Extract failed data to send back to ludlow
# Get all records which haven't been matched to an image + dump to csv
df2 = df.drop(['coords', 'easting', 'northing'], axis=1).loc[lambda df: df.url == 'NaN']
df2.to_csv('ludlow_recs_no_image_match.csv')